# 2. AO Performance Estimation: Resolution

In this section, we will assess the performance of an adaptive optics system by analysing the point spread function (PSF) obtained during closed-loop operation. The PSF is a key indicator of image quality and its characteristics allow us to assess the effectiveness of the AO correction. 

The Full Width at Half Maximum (FWHM) of the PSF is a widely used metric to quantify the sharpness of an image and therefore the performance of the AO system. Ideally, an AO system should produce a narrow PSF, representing a well-corrected, with a FWHM on sky close to a diffraction-limited system:
$$
\text{FWHM}_{dl} \simeq \frac{\lambda}{D_{pupil}}
$$
where $\lambda$ is the wavelength and $D_{pupil}$ is the telescope pupil diameter.

In the following tutorial, we will estimate the FWHM of the closed loop reduced data of *Beta Pegasi* to check the performace of the AO system. We remaind that the data has been collected with PAPYRUS NIR imager camera [C-RED3](https://andor.oxinst.com/products/c-red-series/c-red-3) camera.

Let's start by loading the needed modules and quickly the reduced the closed loop PSF image from the observations as in the previous section.

In [ ]:
!pip uninstall -y oao24
!pip install git+https://github.com/ArcetriAdaptiveOptics/OAO24.git

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.modeling import models, fitting
from oao24.package_data import InfraredExampleData, VisibleExampleData
from oao24 import image_processing

In [ ]:
# backgroud reduction of closed-loop psf data
background_image = InfraredExampleData.get_camera_dark_data()
cl_raw_image_cube = InfraredExampleData.get_close_loop_data_cube()
ao_image = image_processing.make_master_image(cl_raw_image_cube, background_image)

## 2.1 Pixel scale and Diffraction limited FWHM

In this cell, we calculate the expected diffraction-limited PSF FWHM from the nominal parameters for the NIR observations.

We reimnd the following paramters for the T152 telescope and C-RED3 camera:

- **Pupil Diameter** ($D_{pupil}$): 1.52 m (on the Calibration Unit, on sky is reduced by $\sim10\%$)
- **Circual Obstruction** : $33\%$ of M1 size
- **Telescope F-number** ($F/\#$): 23
- **Wavelength** ($\lambda$): 1.55-1.65 µm (on sky)
- **C-Red3 Pixel Size**: 15 µm

  
We can calculate the **pixel scale** in arcseconds per pixel as:

$$
\text{Pixel Scale (arcsec/pixel)} = \frac{\text{Pixel Size (meters)}}{\text{Telescope Focal Length (meters)}} \times \frac{180 \times 3600}{\pi}
$$


While, the FWHM of the diffraction-limited PSF on sky is given by:

$$
\text{FWHM (arcsec)}_{dl} = \frac{\lambda}{D_{pupil}} \times \frac{180 \times 3600}{\pi}
$$


In [12]:
RAD2ARCSEC=180/np.pi*3600
pupil_diameter = 1.52 
wavelength = 1.55e-6   
# Plate scale is typically saved in the FITS image header
# or it can be calibrated using 2 sources of known distance
# Here we use the Papyrus nominal optical design values
# F/# = 23
# The DL PSF size in m on the focal is F/H * wavelength
cred3_pixel_in_meter = 15e-6
f_number =  23
telescope_focal_length = f_number * pupil_diameter
# so the size of the pixel in arcsec is 
pixel_scale_in_arcsec =cred3_pixel_in_meter / telescope_focal_length * RAD2ARCSEC 

# the DL size (=lambda/D) in units of arcsec or pixels are therefore
dl_size_in_arcsec = wavelength / pupil_diameter * RAD2ARCSEC 
dl_size_in_pixels = dl_size_in_arcsec / pixel_scale_in_arcsec

print("C-Red3 pixel scale: %g arcsec/pixel" % pixel_scale_in_arcsec)
print("DL PSF size: %g arcsec" % dl_size_in_arcsec)
print("DL PSF size: %g pixels" % dl_size_in_pixels)

C-Red3 pixel scale: 0.0885003 arcsec/pixel
DL PSF size: 0.210336 arcsec
DL PSF size: 2.37667 pixels


## 2.2 Closed-loop PSF fitting and FWHM estimation

In this step, we select a **region of interest (ROI)** from the science image (i.e. closed-loop image) that focuses on the star image. The aim is to isolate the target star from the surrounding background sky and other sources in the field.

This is important to minimise the influence of noisy sources that of the signal of interest: the ROI should be large enough to capture the full extent of the PSF, but not so large that the background weights the fit.

In [5]:
# We work on a roi of the science image with the star image centered in the field
star_roi = ao_image[240:296, 335:391]
star_roi_cut_index = (29, slice(15, 45))

Once our ROI is defined, we can start simple with a Gaussian fit using the model `Gaussian2D` in **astropy** (see [2D Models](https://docs.astropy.org/en/latest/modeling/predef_models2D.html) for more ditails).

Remind the relation between the gaussian standard deviation $\sigma$ and $\text{FWHM}$: $\text{FWHM} = 2 \sigma \sqrt{2 ln2}$

In [6]:

amp = star_roi.max()
x0 = np.where(star_roi == star_roi.max())[1][0]
y0 = np.where(star_roi == star_roi.max())[0][0]
sigma_x = 2.5/(2*np.sqrt(2*np.log(2)))
sigma_y = sigma_x

model_gauss = models.Gaussian2D(amp, x0, y0, sigma_x, sigma_y)

fitter = fitting.LevMarLSQFitter()

roi_shape = star_roi.shape
y, x = np.mgrid[:roi_shape[0], :roi_shape[1]]

best_fit_gauss = fitter(model_gauss, x, y, z = star_roi)
psf_residual_gauss= best_fit_gauss(x,y)-star_roi

We can define a function like `display_psf_fit`, to show and check the residuals from the Gaussian best fit.

In [7]:
def display_psf_fit(star_roi, best_fit, psf_residual, star_roi_cut_index, label_fit):
    plt.figure()
    plt.imshow(star_roi)
    plt.colorbar()
    plt.title('PSF')

    plt.figure()
    plt.imshow(psf_residual)
    plt.colorbar()
    plt.title('PSF fitting residual (PSF-%s) std %g' % (label_fit, psf_residual.std())) 

    plt.figure()
    plt.plot(best_fit(x,y)[star_roi_cut_index], label=label_fit)
    plt.plot(star_roi[star_roi_cut_index], label='PSF')
    plt.plot(psf_residual[star_roi_cut_index], label='PSF fitting residual')
    plt.legend()

In [ ]:
# displaying gaussian best fit results
display_psf_fit(star_roi, best_fit_gauss, psf_residual_gauss, star_roi_cut_index, 'Gaussian fit')

We can repeat with a `Moffat2D` (see [2D Models](https://docs.astropy.org/en/latest/modeling/predef_models2D.html) for more ditails) model and compare the results with the Gaussian ones. 

It is important to note that the **Moffat model** is commonly used to fit the PSF in **seeing-limited conditions**, as it provides a better representation of the extended wings of the PSF compared to other models like the Gaussian. This makes it particularly effective for fitting the broader distribution caused by atmospheric turbulence, offering a more accurate fit to the PSF's outer regions.

In [ ]:
model_moffat = models.Moffat2D(amp, x0, y0, 2.5)
best_fit_moffat = fitter(model_moffat, x, y, z = star_roi)
psf_residual_moffat= best_fit_moffat(x,y)-star_roi

display_psf_fit(star_roi, best_fit_moffat, psf_residual_moffat, star_roi_cut_index, 'Moffat fit')

We can now compare the Gaussian and Moffat fitting models results:

In [ ]:
plt.figure()
plt.plot(best_fit_gauss(x,y)[star_roi_cut_index], label='Gaussian fit')
plt.plot(best_fit_moffat(x,y)[star_roi_cut_index], label='Moffat fit')
plt.plot(star_roi[star_roi_cut_index], label='PSF')
plt.plot(psf_residual_gauss[star_roi_cut_index], label='Gaussian residual')
plt.plot(psf_residual_moffat[star_roi_cut_index], label='Moffat residual')
plt.legend()

and get the estimated FWHM from the best fitting models:

In [ ]:
###
# convert sigma to FWHM!
# average major and minor axis of the Gaussian
# convert from px to arcsec using the plate scale (given or calibrated)
#
# Advanced: PSF ellipticity can tell you something about anisoplanatism, residual astigmatism, wind direction, ...
###

fwhm_gaussian_fit_px = np.mean(best_fit_gauss.parameters[3:5]*(2*np.sqrt(2*np.log(2))))
fwhm_gaussian_fit_arcsec = fwhm_gaussian_fit_px * pixel_scale_in_arcsec
print('FWHM from Gaussian fit %g arcsec' % (fwhm_gaussian_fit_arcsec))

###
# compute FWHM for the Moffat
# convert from px to arcsec using the plate scale (given or calibrated)
###

fwhm_moffat_fit_px  = best_fit_moffat.parameters[3] * 2
fwhm_moffat_fit_arcsec = fwhm_moffat_fit_px * pixel_scale_in_arcsec
print('FWHM from Moffat fit %g arcsec' % fwhm_moffat_fit_arcsec)
print('DL FWHM %g arcsec' % dl_size_in_arcsec) 


## 2.3 Warnings
As usual, we keep it too easy :-) ! 

We need to keep in mind the following aspects, when we discuss about open and closed-loop PSFs.

- **Selecting the PSF Model**: as the closed-loop data seems to have a high AO correction, the actual PSF looks more like an **Airy disk** diffraction pattern ruther than a Gaussian or a Moffat. Thus, fitting these models, as we did so far, may underestimate or poorly fit the closed loop PSF, especially in the wings, and lead to significant residuals. On the other side, as we mention before, in open-loop condition a Moffat model better fits the broader PSF shape due atmospheric turbulence.

- **Telescope pupil**: so far we considered circular pupil apertures for our fitting models. However, telescope pupils are far from beeing perfectly circular due to the presence of spider that supports the mirror in the optical train, which obscure and block portions of the pupil.

- **PSF ellipcity**: in the fitting models considered in this section, the PSF model is assumed to be radially symmetric. However, the PSF ellipcity can provide valuable insights into several aspects of the optical system and atmosphere. Particularly, the  the elongation and asymmetry of the PSF is related to:
    - **anisoplanatism**, as the angular distance between the guide star and the science target increases;
    - **residual astigmatism**, due imperfect ao correction of the turbulence or NCPA;
    - **wind direction**, that causes elongation or smearing of the PSF due to differential refraction in the atmosphere along the wind direction.

## 2.4 Task and Questions

It's your turn again :-) ! Can you improve the FWHM estimation and minimize the residual by selecting different fitting models and weighting the data?

Particularly,

1. Repeat the fit with Airy diffraction pattern with the **astropy** model `AiryDisk2D` (see [2D Models](https://docs.astropy.org/en/latest/modeling/predef_models2D.html) for more ditails).

2. Advanced: can you improve the fit using a better T152 pupil model? 

3. Repeat the same analysis for the Visible image (Hint: use `VisibleExampleData.get_close_loop_data_cube()` defined in `image_processing.py`and rember to update wavelengths the other parameters for the ORCA Flash 4.0 V2 camera manufactured by Hamamatsu -  6.5um/px, 800nm TBC)

4. Estimate the seeing on open-loop (NB: the open loop examples images in IR and VIS are not good enough, seeing is conventionaly defined in the VIS, need to be rescaled to the observed wavelength).
